<a href="https://colab.research.google.com/github/minhaz1172/Deep-Learning/blob/main/transfer_learning(VGG16).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import dataset from kaggle**

# upload kaggle.json api in colab

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minhaz1172","key":"ae7abd8510b81a9fa7badffec1b23486"}'}

# **Move and set permissson**

In [2]:
import os
import zipfile

# Make directory and move kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set proper permissions
!chmod 600 ~/.kaggle/kaggle.json


# **install kaggle package**

In [3]:
!pip install -q kaggle


# **Find and download the dataset**

In [4]:

!kaggle datasets download -d salader/dogs-vs-cats


Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


# unzip the dataset

In [5]:
# Unzip the dataset (replace the file name if needed)
!unzip -q dogs-vs-cats

replace dogs_vs_cats/test/cats/cat.10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N

N
N


# import necessary Liabraries

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

# **making a class of vgg16**
#  Load the VGG16 model with pretrained ImageNet weights.
# We exclude the top (classification) layers(output layers) since we'll be adding our own.

In [7]:
base_model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

# **Freeze the base model so that its weights do not get updated during the initial training phase**

In [8]:
base_model.trainable=False

In [9]:
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

# Define image dimension and batch size for our image generatio

In [10]:
img_height,img_width=224,224 ## VGG16 expects 224x224 images
batch_size=32  # number of images per gradient update

# Create data generation with augmentation

In [11]:
train_datagen=ImageDataGenerator(
    rescale=1./255, ## Normalize pixel values to [0,1]
    rotation_range=40,  # Randomly rotate images up to 40 degrees
    width_shift_range=0.2, # Randomly shift images horizontally by 20%
    height_shift_range=0.2, # Randomly shift images vertically by 20%
    zoom_range=0.2, #  # Randomly zoom in on images
     horizontal_flip=True, #  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in missing pixels after transformations

)


# create an imageDatagenerator for validation without augmentation(without rescaling

In [12]:
val_datagen=ImageDataGenerator(rescale=1./255)

# **Define the directory paths for training and validation data in your Colab folder**

In [13]:
train_dir='/content/train'
val_dir='/content/test'

# **define the directory path for training and validation data which is in my colab folder**

# Create a data generator that reads images from the training directory,
# applies augmentation, and resizes them to the required dimensions.

In [14]:
train_generator=train_datagen.flow_from_directory(
    train_dir, # directory with training images
    target_size=(img_height,img_width), # resize image with 224 by 224
    batch_size=batch_size, #number
    class_mode='categorical'
)


Found 20000 images belonging to 2 classes.


# create a data generator for validation data(no augmentation is applied)

In [15]:
validation_generator=val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5000 images belonging to 2 classes.


# **COMPILE THE MODEL**

# Determine the number of classes from the training generator.

In [16]:
num_classes=len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

Number of classes: 2


# Create a sequential mode and add layers

In [17]:
model=Sequential()
model.add(base_model)  # Add the VGG16 convolutional base.
model.add(Flatten()) # Flatten the 3D feature maps to 1D feature vectors.
model.add(Dense(256,activation='relu')) # add a fully connected dense layer
model.add(Dropout(0.5)) # add dropout for regularization to help prevent overfitting.
model.add(Dense(num_classes,activation='softmax')) # add a final output layer



# compile the model

In [18]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Set the number of epochs for the initial training phase.


In [19]:
initial_epochs = 10

# **Train the model using the training generator and validate on the validation generator**

In [22]:
history=model.fit(
    train_generator,
    steps_per_epoch = min(500 // batch_size, train_generator.samples // batch_size), # takin oly 500 images to train
    epochs=initial_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples //batch_size
                  )

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.8707 - loss: 0.3141 - val_accuracy: 0.9115 - val_loss: 0.2148
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.8522 - loss: 0.3553 - val_accuracy: 0.9052 - val_loss: 0.2180
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.8494 - loss: 0.3286 - val_accuracy: 0.9129 - val_loss: 0.2262
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.8433 - loss: 0.3363 - val_accuracy: 0.9079 - val_loss: 0.2160
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.8290 - loss: 0.4005 - val_accuracy: 0.9165 - val_loss: 0.2004
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.8681 - loss: 0.2982 - val_accuracy: 0.9081 - val_loss: 0.2161
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.8495 - loss: 0.4385 - val_accuracy: 0.9183 - val_loss: 0.2053
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.8516 - loss: 0.3424 - val_accuracy: 0.9161 - val_loss:

# FINE-TUNE THE MODEL (UNFREEZE SOME LAYERS OF VGG16)
# Unfreeze the entire base model to prepare for fine-tuning.

In [23]:
base_model.trainable= True

# For fine-tuning, we freeze the earlier layers and only train the last few layers of VGG16.
# This prevents losing the general features learned from ImageNet.

In [24]:
# as vgg16 has 16 convlayers,3 fully conneted layer
fine_tune_at=len(base_model.layers)-4 ## Unfreeze the last 4 layers only.
for layer in base_model.layers[:fine_tune_at]:      # freeze the layers before last 4 four layers
  layer.trainable=False

# Recompile the model with a very low learning rate to avoid large weight updates during fine-tuning.

In [25]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])